In [1]:
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import requests
from sqlalchemy import create_engine
from config import trefle_key
token = trefle_key
# Document Resource :https://docs.trefle.io/docs/examples/snippets

In [3]:
# Create an URL 
url = f"https://trefle.io/api/v1/plants?token={token}"


In [4]:
# Need to get data on all plants and species and save it as a DB 
all_plants = requests.get(url)
plants_list = all_plants.json()['data']


In [9]:
# plants_list

In [11]:
# all_plants.json()

In [13]:
# Need to get data on all species and species and save it as a DB 
species_url = url = f"https://trefle.io/api/v1/species?token={token}"
all_species = requests.get(species_url)
species_list = all_plants.json()['data']
# species_list

In [15]:
# Need to get data on all edible plants and species and save it as a DB 
edible_url = f"https://trefle.io/api/v1/plants?filter_not%5Bedible_part%5D=null&token={token}"
all_edible = requests.get(edible_url)
edible_list = all_plants.json()['data']
# edible_list

In [18]:
# Get all Tallest trees
tall_tree_url = f"https://trefle.io/api/v1/plants?filter_not%5Bmaximum_height_cm%5D=null&filter%5Bligneous_type%5D=tree&order%5Bmaximum_height_cm%5D=desc&token={token}"
all_tall_tree = requests.get(tall_tree_url)
tall_tree_list = all_tall_tree.json()['data']
# tall_tree_list

In [20]:
# Get all the Countries/zones
zones_url = f"https://trefle.io/api/v1/distributions?token={token}"
zones_list = requests.get(zones_url).json()['data']
zones_list

[{'id': 446,
  'name': 'Abkhaziya',
  'slug': 'tcs-ab',
  'tdwg_code': 'TCS-AB',
  'tdwg_level': 4,
  'species_count': 0,
  'links': {'self': '/api/v1/distributions/tcs-ab',
   'plants': '/api/v1/distributions/tcs-ab/plants',
   'species': '/api/v1/distributions/tcs-ab/species'},
  'parent': {'id': 272,
   'name': 'Transcaucasus',
   'slug': 'tcs',
   'tdwg_code': 'TCS',
   'tdwg_level': 3,
   'species_count': 5227,
   'links': {'self': '/api/v1/distributions/tcs',
    'plants': '/api/v1/distributions/tcs/plants',
    'species': '/api/v1/distributions/tcs/species'}},
  'children': []},
 {'id': 694,
  'name': 'Acre',
  'slug': 'bzn-ac',
  'tdwg_code': 'BZN-AC',
  'tdwg_level': 4,
  'species_count': 0,
  'links': {'self': '/api/v1/distributions/bzn-ac',
   'plants': '/api/v1/distributions/bzn-ac/plants',
   'species': '/api/v1/distributions/bzn-ac/species'},
  'parent': {'id': 78,
   'name': 'Brazil North',
   'slug': 'bzn',
   'tdwg_code': 'BZN',
   'tdwg_level': 3,
   'species_count': 

In [ ]:
requests.get(zones_url).json()

In [ ]:
zone_test = requests.get(zones_url).json()
# zone_test

In [ ]:
# Get all Family
family_url = "https://trefle.io/api/v1/families/?token={token}"
family_list = requests.get(family_url).json()['data']
# family_list

In [ ]:
# Get all genus
genus_url = f"https://trefle.io/api/v1/genus?token={token}"
genus_list = requests.get(genus_url).json()['data']

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.trefle
collection = db.plants

In [ ]:
collection.insert_many(plants_list)

In [ ]:
collection = db.species
collection.insert_many(species_list)

In [ ]:
collection = db.edible_plants
collection.insert_many(edible_list)

In [ ]:
collection = db.tall_tree
collection.insert_many(tall_tree_list)

In [ ]:
collection = db.country_zones
collection.insert_many(zones_list)

In [ ]:
collection = db.genus
collection.insert_many(genus_list)

In [ ]:
collection =db.family
collection.insert_many(family_list)